# Assignment 1, problem 4
Nicholai L'Esperance

## Imports

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error as mse

import numpy as np
import pandas as pd
import matplotlib as mpl

In [3]:
day_data = pd.read_csv('day.csv')
print(day_data.shape)
day_data.head()

(731, 16)


,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


## Feature Selection

Before deciding on my feature set, I create a correlation matrix, looking for variables that are very strongly correlated (as I might not want to include both as features in my analysis). 

In [4]:
corr = day_data.corr()
corr.style.background_gradient(vmin=-1, vmax=1, cmap=mpl.cm.PiYG)

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
instant,1.000000,0.412224,0.866025,0.496702,0.016145,-0.000016,-0.004337,-0.021477,0.150580,0.152638,0.016375,-0.112620,0.275255,0.659623,0.628830
season,0.412224,1.000000,-0.001844,0.831440,-0.010537,-0.003080,0.012485,0.019211,0.334315,0.342876,0.205445,-0.229046,0.210399,0.411623,0.406100
yr,0.866025,-0.001844,1.000000,-0.001792,0.007954,-0.005461,-0.002013,-0.048727,0.047604,0.046106,-0.110651,-0.011817,0.248546,0.594248,0.566710
mnth,0.496702,0.831440,-0.001792,1.000000,0.019191,0.009509,-0.005901,0.043528,0.220205,0.227459,0.222204,-0.207502,0.123006,0.293488,0.279977
holiday,0.016145,-0.010537,0.007954,0.019191,1.000000,-0.101960,-0.253023,-0.034627,-0.028556,-0.032507,-0.015937,0.006292,0.054274,-0.108745,-0.068348
weekday,-0.000016,-0.003080,-0.005461,0.009509,-0.101960,1.000000,0.035790,0.031087,-0.000170,-0.007537,-0.052232,0.014282,0.059923,0.057367,0.067443
workingday,-0.004337,0.012485,-0.002013,-0.005901,-0.253023,0.035790,1.000000,0.061200,0.052660,0.052182,0.024327,-0.018796,-0.518044,0.303907,0.061156
weathersit,-0.021477,0.019211,-0.048727,0.043528,-0.034627,0.031087,0.061200,1.000000,-0.120602,-0.121583,0.591045,0.039511,-0.247353,-0.260388,-0.297391
temp,0.150580,0.334315,0.047604,0.220205,-0.028556,-0.000170,0.052660,-0.120602,1.000000,0.991702,0.126963,-0.157944,0.543285,0.540012,0.627494
atemp,0.152638,0.342876,0.046106,0.227459,-0.032507,-0.007537,0.052182,-0.121583,0.991702,1.000000,0.139988,-0.183643,0.543864,0.544192,0.631066


Looking at the correlations, we see that much of the date information correlates with itself (of course). I do not want to include the date information in my features, as I want my model to be based just on weekday and weather information. However, I think including season is a good idea, especially noting its correlation with the number of bikes rented. The other thing I note, is a near perfect correlation between temp and atemp, so I will leave atemp out of my model. The other thing I notice, is that weekday, workingday, and holiday are all strongly connected. Weekday is always equal to workingday except when holiday is 1. For that reason, I am not going to include workingday in my model either, as it does not contain additional information.

Now, we break this data up into the independent and dependent variables.  For the dependent variable, I am just going to predict the total bike rentals (cnt).

In [5]:
X = day_data[['season', 'holiday', 'weekday', 'weathersit', 'temp', 'hum', 'windspeed']]
y = day_data['cnt']

Next, I break the X and y data up into training and testing datasets. For this, I choose an 90/10 split.

## Train/Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

## Model 1: Linear Regression

I will use the sklearn pipline and standard scalar to build this model. The scalar removes the mean of each feature, and scales it by the variance. This makes it so each feature can contribute equally to the model, and units of one feature do not overwhelm other features.  

I am using 4-fold cross validation.

In [7]:
lr = LinearRegression()

lr_pipeline = make_pipeline(StandardScaler(), lr)

#### CV Score
Note, sklearn uses a 'negative' mean squared error. We easily convert this to MSE by multiplying by -1.

In [8]:
lr_scores = cross_val_score(lr_pipeline, X_train, y_train, cv=4, scoring='neg_mean_squared_error') * -1
print(f'Average CV MSE Score: {np.mean(lr_scores)}')

Average CV MSE Score: 1769715.4675387586


#### MSE

In [9]:
x_scaler = StandardScaler()
x_scaler.fit(X_train)

lr.fit(x_scaler.transform(X_train), y_train)

lr_predict = lr.predict(x_scaler.transform(X_test))
lr_error = lr_predict - y_test

lr_mae = np.mean(lr_error.abs())
lr_mse = np.mean(lr_error ** 2)

print(f'Linear Regression MAE: {lr_mae}')
print(f'Linear Regression MSE: {lr_mse}')

Linear Regression MAE: 1312.1204580441056
Linear Regression MSE: 2292231.332262654


## Model 2: Ridge Regression

This process will follow the same steps as the previous model, so we can directly compare the results. Ridge operates very much like the regular linear regression, but includes a penalty to prevent coefficients from getting too large. This penalty coefficient is named alpha; I am setting this to 5 for this model.

In [10]:
rr = Ridge(alpha=5)

rr_pipeline = make_pipeline(StandardScaler(), rr)

#### CV Score

In [11]:
rr_scores = cross_val_score(rr_pipeline, X_train, y_train, cv=4, scoring='neg_mean_squared_error') * -1
print(f'Average CV MSE Score: {np.mean(rr_scores)}')

Average CV MSE Score: 1768889.9110753355


#### MSE

In [12]:
x_scaler = StandardScaler()
x_scaler.fit(X_train)

rr.fit(x_scaler.transform(X_train), y_train)

rr_predict = rr.predict(x_scaler.transform(X_test))
rr_error = rr_predict - y_test

rr_mae = np.mean(rr_error.abs())
rr_mse = np.mean(rr_error ** 2)

print(f'Linear Regression MAE: {rr_mae}')
print(f'Linear Regression MSE: {rr_mse}')

Linear Regression MAE: 1312.7313116672149
Linear Regression MSE: 2291404.0558082326


This model is performing almost exactly the same as the previous model, slightly worse. In this case, we are probably reducing the amplitude of the most important feature. However, perhaps we are not overfitting as much as guessed, and the value of alpha should be lower than I chose. The effect of alpha can be seen directly, comparing the coefficients of the two models:

In [28]:
print('feature       LR Coefficient       RR Coefficient')
print('------------------------------------------------------')
for lr_coeff, rr_coeff, feature in zip(lr.coef_, rr.coef_, X.columns):
    feat = f'{feature}:'
    print(f'{feat.ljust(12)} {str(lr_coeff).ljust(20)} {rr_coeff}')

feature       LR Coefficient       RR Coefficient
------------------------------------------------------
season:      441.7430532126713    440.2642456422432
holiday:     -91.85948633651469   -91.52789806846772
weekday:     113.40495782694373   112.92599540851641
weathersit:  -211.30971402041877  -214.0941164503526
temp:        1037.2572919108604   1029.172496771858
hum:         -336.399313745584    -330.31721470199784
windspeed:   -291.0492698459857   -288.8241768557817


## Model 3: Polynomial Regression, two degress

To perform a polynomial regression using sklearn, we transform our features into polynomial features of degree n. In this case, I am choosing a degree of n=2.

In [31]:
degrees = 2

pr = LinearRegression()

poly = PolynomialFeatures(degree=degrees)
poly_train = poly.fit_transform(X_train)

#### CV Score

In [32]:
pr_scores = cross_val_score(pr, poly_train, y_train, cv=4, scoring='neg_mean_squared_error') * -1
print(f'Average CV MSE Score: {np.mean(pr_scores)}')

Average CV MSE Score: 13009649.679693274


I am not sure why the CV MSE is so much higher than the previous two models, given that the actual error on the test set below is lower than the previous two models.

#### MSE

In [33]:
pr.fit(poly_train, y_train)

pr_predict = pr.predict(poly.transform(X_test))
pr_error = pr_predict - y_test

pr_mae = np.mean(pr_error.abs())
pr_mse = np.mean(pr_error ** 2)

print(f'Linear Regression MAE: {pr_mae}')
print(f'Linear Regression MSE: {pr_mse}')

Linear Regression MAE: 1240.998652867586
Linear Regression MSE: 1930412.4413201716


Despite having a significantly higher CV MSE score than the previous two models, this third model has a lower MAE and MSE than both of the previous models.

## Model 4: Polynomial Regression, four degress

To perform a polynomial regression using sklearn, we transform our features into polynomial features of degree n. In this case, I am choosing a degree of n=2.

In [34]:
degrees = 4

pr = LinearRegression()

poly = PolynomialFeatures(degree=degrees)
poly_train = poly.fit_transform(X_train)

#### CV Score

In [35]:
pr_scores = cross_val_score(pr, poly_train, y_train, cv=4, scoring='neg_mean_squared_error') * -1
print(f'Average CV MSE Score: {np.mean(pr_scores)}')

Average CV MSE Score: 9.85851444337936e+25


I am not sure why the CV MSE is so much higher than the previous two models, given that the actual error on the test set below is lower than the previous two models.

#### MSE

In [36]:
pr.fit(poly_train, y_train)

pr_predict = pr.predict(poly.transform(X_test))
pr_error = pr_predict - y_test

pr_mae = np.mean(pr_error.abs())
pr_mse = np.mean(pr_error ** 2)

print(f'Linear Regression MAE: {pr_mae}')
print(f'Linear Regression MSE: {pr_mse}')

Linear Regression MAE: 1853.475681759992
Linear Regression MSE: 8807973.05864682


Despite having a significantly higher CV MSE score than the previous two models, this third model has a lower MAE and MSE than both of the previous models.